# Ipysurfer Examples

## Basic Visualization

Initialize a basic visualization session and plot a brain mesh. For this purpose we need to provide subject information, e.g. `subject_id`, `subjects_dir`; which type of freesurfer surface mesh we would like to plot, e.g. 'pialed' or 'inflated'; and which hemisphere to plot or whether to plot both.

## Pial Surface, Both Hemispheres on a Plot

In [73]:
import os.path as path

import mne
from ipysurfer import Brain

In [74]:
data_path = mne.datasets.sample.data_path()

subject_id = 'sample'
subjects_dir = path.join(data_path, 'subjects')

# show both hemispheres on a plot
hemi = 'both'
surf = 'pial'

# Call the Brain object constructor with
# parameters to initialize the visualization session.
brain = Brain(subject_id, hemi, surf,
              size=300, subjects_dir=subjects_dir)
brain.show()

## Inflated Surface, Separated Hemispheres

In [75]:
surf = 'inflated'
# show both hemispheres on separate plots
hemi = 'split'

# Call the Brain object constructor with
# parameters to initialize the visualization session.
brain = Brain(subject_id, hemi, surf,
              size=300, subjects_dir=subjects_dir)
brain.show()

## Activation Data Visualization

In this example it is shown how to add activation data to a plot. By default, color bar and input boxes for fmin, fmid and fmax control parameters of a color map are attached to the plot.

In this example it is shown how to add activation data to a plot. By default, color bar and input boxes for `fmin`, `fmid` and `fmax` control parameters of a color map are attached to the plot.

A colormap may be sequential or divergent, the latter has defined `center` parameter. The meanings of fmin, fmid and fmax are different for sequential and divergent colormaps. A sequential colormap is characterised by 
```
[fmin, fmid, fmax]
```
where `fmin` and `fmax` define the edges of the colormap and `fmid` will be the value mapped to the center of the originally chosen colormap. A divergent colormap is characterised by
```
[center-fmax, center-fmid, center-fmin, center, center+fmin, center+fmid, center+fmax]
```
i.e., values between center-fmin and center+fmin will not be shown while `center-fmid` will map to the middle of the first half of the original colormap and `center-fmid` to the middle of the second half.

In [76]:
act_data = path.join(data_path, 'MEG/sample/sample_audvis-meg-eeg')

# read mne provided example of data
stc = mne.read_source_estimate(act_data)

# select which hemisphere to plot
hemi = 'lh'
# select data that corresponds to the 'lh' hemisphere for the selected time moment
hemi_data = stc.data[:len(stc.vertices[0]), 10]
hemi_vertices = stc.vertices[0]

fmin = stc.data.min()
fmax = stc.data.max()

In [77]:
brain_data = Brain(subject_id, hemi, surf,
                   size=300, subjects_dir=subjects_dir)

brain_data.add_data(hemi_data, fmin=fmin, hemi=hemi,
                    fmax=fmax, colormap='hot',
                    vertices=hemi_vertices)
brain_data.show()

Color bar controls can be disabled as in the following example.

In [78]:
brain_data = Brain(subject_id, hemi, surf,
                   size=300, subjects_dir=subjects_dir)

# color bar controls are attached by default, so we need to change parameter value
brain_data.add_data(hemi_data, fmin=fmin, hemi=hemi,
                    fmax=fmax, colormap='hot',
                    vertices=hemi_vertices, colorbar=False)
brain_data.show()

## Activation Data Visualization for Different Time Moments

Use `TimeViewer` class to append time viewer widget to the plot.

In [79]:
from ipysurfer import TimeViewer

In [80]:
act_data = path.join(data_path,
                     'MEG/sample/sample_audvis-meg-eeg')

# read mne provided example of data
stc = mne.read_source_estimate(act_data)

# select which hemisphere to plot
hemi = 'rh'

# select data that corresponds to 'lh' hemisphere for each moment of time
hemi_data = stc.data[:len(stc.vertices[0]), :]
hemi_vertices = stc.vertices[0]

# we will need to provide list of time points and select initial time
hemi_times = stc.times
hemi_initial_time = 0.09

# time label formating string to be shown
hemi_time_label = 'time=%0.2f ms'

fmin = 5
fmid = 7
fmax = 28

In [81]:
brain_data = Brain(subject_id, hemi, surf,
                   size=400, subjects_dir=subjects_dir)

brain_data.add_data(hemi_data, hemi=hemi, fmin=fmin, 
                    fmid=fmid, fmax=fmax, colormap='hot',
                    initial_time=hemi_initial_time,
                    time=hemi_times, vertices=hemi_vertices,
                    colorbar=False, time_label=hemi_time_label)

time_viewer = TimeViewer(brain_data)
time_viewer.show()

We can show color bar and time viewer widgets together

In [56]:
brain_data = Brain(subject_id, hemi, surf,
                   size=400, subjects_dir=subjects_dir)
brain_data.add_data(hemi_data, fmin=fmin, hemi=hemi,
                    fmax=fmax, colormap='hot',
                    initial_time=hemi_initial_time,
                    time=hemi_times, vertices=hemi_vertices,
                    colorbar=True, time_label=hemi_time_label)

time_viewer = TimeViewer(brain_data)
time_viewer.show()

## Plot MNE Source Estimates Signed Data

In [82]:
from mne import read_source_estimate

from ipysurfer import plot_source_estimates

In [83]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse

# Process MEG data

data_path = sample.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'

raw = mne.io.read_raw_fif(raw_fname)  # already has an average reference
events = mne.find_events(raw, stim_channel='STI 014')

event_id = dict(aud_r=1)  # event trigger and conditions
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)
raw.info['bads'] = ['MEG 2443', 'EEG 053']
picks = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                       exclude='bads')
baseline = (None, 0)  # means from the first instant to t = 0
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, reject=reject)

###############################################################################
# Compute regularized noise covariance
# ------------------------------------
#
# For more details see :ref:`tut_compute_covariance`.

noise_cov = mne.compute_covariance(
    epochs, tmax=0., method=['shrunk', 'empirical'], verbose=True)

###############################################################################
# Compute the evoked response
# ---------------------------
# Let's just use MEG channels for simplicity.

evoked = epochs.average().pick_types(meg=True)
del epochs  # to save memory

###############################################################################
# Inverse modeling: MNE/dSPM on evoked and raw data
# -------------------------------------------------

# Read the forward solution and compute the inverse operator
fname_fwd = data_path + '/MEG/sample/sample_audvis-meg-oct-6-fwd.fif'
fwd = mne.read_forward_solution(fname_fwd)

# make an MEG inverse operator
info = evoked.info
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)
del fwd

# Compute inverse solution
# ------------------------

method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(evoked, inverse_operator, lambda2,
                    method=method, pick_ori='normal')

Opening raw data file /home/okozyn/mne_data/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Current compensation grade : 0
319 events found
Event IDs: [ 1  2  3  4  5 32]
72 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 3)
4 projection items activated
Loading data for 72 events and 106 original time points ...
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 1711']
    Rejecting  epoch based on EOG : ['EOG 061']
    Reje

In [84]:
# stc is an instance of MNE.SourceEstimates class
# that holds signed data 
fig = plot_source_estimates(stc, subject=subject_id, 
                            surface='inflated', hemi='both', 
                            colormap='auto', smoothing_steps=10,
                            time_unit='s', time_label='auto',
                            transparent=False, alpha=1.0,
                            time_viewer=True, clim='auto', size=400,
                            subjects_dir=subjects_dir, background="black",
                            initial_time=0.09, colorbar=True)

## Creating a Screenshot

In [85]:
# stc is an instance of MNE.SourceEstimates class
# that holds signed data
fig = plot_source_estimates(stc, subject=subject_id, 
                            surface='inflated', hemi='both', 
                            colormap='auto', smoothing_steps=10,
                            time_unit='s', time_label='auto',
                            transparent=False, alpha=1.0,
                            time_viewer=True, clim='auto', size=400,
                            subjects_dir=subjects_dir, background="black",
                            initial_time=0.09, colorbar=True)